In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

In [2]:
data = pd.read_csv(r"C:\Users\HomePC\Downloads\Dataset.xlsx - Sheet1.csv")

In [3]:
cols_to_extract = ['PurchaseHistory', 'SubscriptionDetails', 'ServiceInteractions', 'PaymentHistory', 
                   'WebsiteUsage', 'ClickstreamData', 'EngagementMetrics', 'Feedback', 'MarketingCommunication']

In [4]:
import ast

def convert_string(col: str) -> any:
    if isinstance(col, str): 
        return ast.literal_eval(col)
    return col

extracted_tables = {}

for col in cols_to_extract:
    data[col] = data[col].apply(convert_string)
    normalized = pd.DataFrame()

    if isinstance(data[col].iloc[0], list):
        temp = data[['CustomerID', col]].explode(col)
        normalized = pd.json_normalize(temp[col])
        normalized['CustomerID'] = temp['CustomerID'].values
    elif isinstance(data[col].iloc[0], dict):
        normalized = pd.json_normalize(data[col])
        normalized['CustomerID'] = data['CustomerID'].values

    extracted_tables[col] = normalized

In [33]:
PurchaseHistory, SubscriptionDetails, ServiceInteractions, PaymentHistory, WebsiteUsage, \
ClickstreamData, EngagementMetrics, Feedback, MarketingCommunication = [extracted_tables[k] for k in extracted_tables.keys()]

In [38]:
ServiceInteractions

,Type,Date,CustomerID
0,Call,2019-09-26,1001
1,Chat,2021-07-25,1001
2,Email,2020-04-13,1001
3,Chat,2020-11-15,1001
4,Call,2020-01-05,1002
...,...,...,...
254248,Call,2022-04-06,13483
254249,Call,2021-01-31,13483
254250,Chat,2021-11-12,13483
254251,Call,2022-06-10,13483


In [34]:
for key, values in extracted_tables.items():
    print(key, values.shape)

PurchaseHistory (68628, 4)
SubscriptionDetails (12483, 7)
ServiceInteractions (254253, 3)
PaymentHistory (37449, 3)
WebsiteUsage (12483, 3)
ClickstreamData (319616, 4)
EngagementMetrics (12483, 3)
Feedback (12483, 3)
MarketingCommunication (68762, 4)


## Feature Engineering

Purchase Behavior Features

**Idea:** Customers who spend more and buy consistently are less likely to churn.

We calculate:
- Total/average spending (loyalty indicator)
- Purchase consistency (stable vs erratic buyers)
- Product diversity (engaged vs single-product users)

In [35]:
PurchaseHistory = PurchaseHistory.groupby('CustomerID').agg({
    'Value': 'sum',
    'Frequency': 'sum',
    'Product': 'nunique'
}).reset_index()
PurchaseHistory.columns = ['CustomerID', 'total_purchase_value', 'total_frequency', 'product_diversity']

In [36]:
PurchaseHistory

,CustomerID,total_purchase_value,total_frequency,product_diversity
0,1001,3994.72,38,7
1,1002,2844.35,4,3
2,1003,1866.52,14,3
3,1004,1378.64,28,5
4,1005,2425.05,39,6
...,...,...,...,...
12478,13479,1196.56,14,3
12479,13480,710.57,1,1
12480,13481,5154.42,63,10
12481,13482,6055.16,58,9


### Subscription Lifecycle Features

**Idea:** Long-term subscribers with active accounts are less likely to churn.

We calculate:
- How long they've been subscribed (tenure)
- Whether subscription is still active
- Subscription plan type

In [ ]:
SubscriptionDetails['Start_Date'] = pd.to_datetime(SubscriptionDetails['Start_Date'])
SubscriptionDetails['End_Date'] = pd.to_datetime(SubscriptionDetails['End_Date'])
SubscriptionDetails['subscription_duration_days'] = (SubscriptionDetails['End_Date'] - SubscriptionDetails['Start_Date']).dt.days
SubscriptionDetails['subscription_age_days'] = (SubscriptionDetails['End_Date'].max() - SubscriptionDetails['Start_Date']).dt.days
SubscriptionDetails['is_active'] = (SubscriptionDetails['End_Date'] > SubscriptionDetails['End_Date'].max()).astype(int)
SubscriptionDetails = SubscriptionDetails[['CustomerID', 'Plan', 'subscription_duration_days', 'subscription_age_days', 'is_active']]

In [38]:
SubscriptionDetails

,CustomerID,Plan,subscription_duration_days,subscription_age_days,is_active
0,1001,Express,871,937,0
1,1002,Pro,290,529,0
2,1003,Essential,319,1184,0
3,1004,Smart,803,1083,0
4,1005,Basic,580,633,0
...,...,...,...,...,...
12478,13479,Essential,745,1296,0
12479,13480,Flex,18,22,0
12480,13481,Deluxe,20,546,0
12481,13482,Gold,484,894,0


### Service Interaction Patterns

**Idea:** Frequent support contacts (especially calls) may signal dissatisfaction.

We calculate:
- Total interactions
- Calculate last interaction date

In [39]:
ServiceInteractions['Date'] = pd.to_datetime(ServiceInteractions['Date'])
ServiceInteractions = ServiceInteractions.groupby('CustomerID').agg({
    'Date': 'max',
    'Type': 'count'
}).reset_index()
ServiceInteractions.columns = ['CustomerID', 'last_interaction_date', 'total_interactions']

In [40]:
ServiceInteractions

,CustomerID,last_interaction_date,total_interactions
0,1001,2021-07-25,4
1,1002,2022-12-13,19
2,1003,2022-01-04,3
3,1004,2022-11-10,59
4,1005,2022-12-19,10
...,...,...,...
12478,13479,2022-10-09,10
12479,13480,2022-11-05,3
12480,13481,2022-12-08,26
12481,13482,2022-11-15,13


### Payment Reliability Features

**Idea:** Late payments indicate financial stress or dissatisfaction.

We calculate:
- Total late payments
- Late payment rate (percentage of payments that are late)
- Payment risk score (combined metric)

In [41]:
PaymentHistory = PaymentHistory.groupby('CustomerID').agg({
    'Late_Payments': 'sum',
    'Method': 'count'
}).reset_index()
PaymentHistory.columns = ['CustomerID', 'total_late_payments', 'payment_count']
PaymentHistory['late_payment_rate'] = PaymentHistory['total_late_payments'] / (PaymentHistory['payment_count'] + 1)
PaymentHistory['payment_risk_score'] = PaymentHistory['total_late_payments'] * PaymentHistory['late_payment_rate']

In [42]:
PaymentHistory

,CustomerID,total_late_payments,payment_count,late_payment_rate,payment_risk_score
0,1001,40,3,10.00,400.00
1,1002,10,3,2.50,25.00
2,1003,8,3,2.00,16.00
3,1004,79,3,19.75,1560.25
4,1005,2,3,0.50,1.00
...,...,...,...,...,...
12478,13479,3,3,0.75,2.25
12479,13480,6,3,1.50,9.00
12480,13481,83,3,20.75,1722.25
12481,13482,67,3,16.75,1122.25


### Digital Engagement Features

**Idea:** Active website users are more engaged and less likely to leave.

We calculate:
- Page views and time spent
- Engagement ratio (time per page)
- Engagement intensity (overall activity level)

In [43]:
WebsiteUsage = WebsiteUsage.groupby('CustomerID').agg({
    'PageViews': 'sum',
    'TimeSpent(minutes)': 'sum'
}).reset_index()
WebsiteUsage['engagement_ratio'] = WebsiteUsage['TimeSpent(minutes)'] / (WebsiteUsage['PageViews'] + 1)
WebsiteUsage['engagement_intensity'] = WebsiteUsage['PageViews'] * WebsiteUsage['TimeSpent(minutes)']

In [44]:
WebsiteUsage

,CustomerID,PageViews,TimeSpent(minutes),engagement_ratio,engagement_intensity
0,1001,49,15,0.300000,735
1,1002,100,9,0.089109,900
2,1003,1,97,48.500000,97
3,1004,25,31,1.192308,775
4,1005,77,51,0.653846,3927
...,...,...,...,...,...
12478,13479,70,57,0.802817,3990
12479,13480,71,66,0.916667,4686
12480,13481,96,1,0.010309,96
12481,13482,63,2,0.031250,126


### Clickstream Behavior Features

**Idea:** Specific actions (adding to cart, searching) show purchase intent.

We calculate:
- Total actions taken
- Cart conversion rate (add-to-cart actions)
- Search intensity (exploration behavior)
- Page diversity (breadth of interest)

In [18]:
plus = lambda x: x + 1

plus(5)

6

In [45]:
ClickstreamData = ClickstreamData.groupby('CustomerID').agg({
    'Action': ['count', lambda x: (x == 'Add to Cart').sum(), lambda x: (x == 'Search').sum(), lambda x: (x == 'Click').sum()],
    'Page': 'nunique'
}).reset_index()
ClickstreamData.columns = ['CustomerID', 'total_actions', 'add_to_cart_count', 'search_count', 'click_count', 'unique_pages']
ClickstreamData['cart_conversion_rate'] = ClickstreamData['add_to_cart_count'] / (ClickstreamData['total_actions'] + 1)
ClickstreamData['search_intensity'] = ClickstreamData['search_count'] / (ClickstreamData['total_actions'] + 1)
ClickstreamData['page_diversity'] = ClickstreamData['unique_pages'] / (ClickstreamData['total_actions'] + 1)

In [46]:
ClickstreamData

,CustomerID,total_actions,add_to_cart_count,search_count,click_count,unique_pages,cart_conversion_rate,search_intensity,page_diversity
0,1001,24,8,12,4,13,0.320000,0.480000,0.520000
1,1002,24,8,7,9,13,0.320000,0.280000,0.520000
2,1003,12,2,7,3,7,0.153846,0.538462,0.538462
3,1004,47,15,16,16,14,0.312500,0.333333,0.291667
4,1005,30,17,4,9,12,0.548387,0.129032,0.387097
...,...,...,...,...,...,...,...,...,...
12478,13479,6,4,1,1,6,0.571429,0.142857,0.857143
12479,13480,9,3,3,3,8,0.300000,0.300000,0.800000
12480,13481,26,10,11,5,9,0.370370,0.407407,0.333333
12481,13482,38,7,15,16,12,0.179487,0.384615,0.307692


### Engagement Frequency Features

**Idea:** Daily users are more engaged than monthly users.

We convert frequency categories to numeric scores:
- Daily = 30 (highest engagement)
- Weekly = 4
- Monthly = 1 (lowest engagement)

In [47]:
frequency_map = {'Daily': 30, 'Weekly': 4, 'Monthly': 1}
EngagementMetrics = EngagementMetrics.groupby('CustomerID').agg({
    'Logins': 'sum',
    'Frequency': 'first'
}).reset_index()
EngagementMetrics['frequency_score'] = EngagementMetrics['Frequency'].map(frequency_map)
EngagementMetrics['engagement_score'] = EngagementMetrics['Logins'] * EngagementMetrics['frequency_score']
EngagementMetrics = EngagementMetrics[['CustomerID', 'Logins', 'frequency_score', 'engagement_score']]

In [48]:
EngagementMetrics

,CustomerID,Logins,frequency_score,engagement_score
0,1001,19,4,76
1,1002,9,4,36
2,1003,19,1,19
3,1004,4,30,120
4,1005,12,4,48
...,...,...,...,...
12478,13479,22,30,660
12479,13480,25,4,100
12480,13481,9,1,9
12481,13482,2,1,2


### Sentiment & Feedback Features

**Idea:** Negative feedback and low ratings predict churn.

We calculate:
- Average rating
- Comment length (longer = more invested)
- Sentiment flags (positive/negative)

In [49]:
Feedback = Feedback.groupby('CustomerID').agg({
    'Rating': 'mean',
    'Comment': lambda x: x.str.len().mean()
}).reset_index()
Feedback.columns = ['CustomerID', 'avg_rating', 'avg_comment_length']
Feedback['sentiment_score'] = Feedback['avg_rating'] / 5
Feedback['is_negative'] = (Feedback['avg_rating'] <= 2).astype(int)
Feedback['is_positive'] = (Feedback['avg_rating'] >= 4).astype(int)

In [50]:
Feedback

,CustomerID,avg_rating,avg_comment_length,sentiment_score,is_negative,is_positive
0,1001,1.0,96.0,0.2,1,0
1,1002,2.0,108.0,0.4,1,0
2,1003,4.0,72.0,0.8,0,1
3,1004,1.0,78.0,0.2,1,0
4,1005,3.0,99.0,0.6,0,0
...,...,...,...,...,...,...
12478,13479,2.0,37.0,0.4,1,0
12479,13480,3.0,102.0,0.6,0,0
12480,13481,5.0,134.0,1.0,0,1
12481,13482,5.0,113.0,1.0,0,1


## Marketing Response Features

**Idea:** Customers who ignore emails are disengaging.

We calculate:
- Email open rate
- Click rate
- Click-through rate (clicks per open)
- Overall marketing engagement

In [55]:
MarketingCommunication

,Email_Sent,Email_Opened,Email_Clicked,CustomerID
0,2019-10-17,2022-01-12,2022-11-27,1001
1,2019-10-17,2022-01-12,2022-11-27,1001
2,2019-10-17,2022-01-12,2022-11-27,1001
3,2019-10-17,2022-01-12,2022-11-27,1001
4,2019-10-17,2022-01-12,2022-11-27,1001
...,...,...,...,...
68757,2021-02-08,2021-09-18,2022-04-11,13483
68758,2021-02-08,2021-09-18,2022-04-11,13483
68759,2021-02-08,2021-09-18,2022-04-11,13483
68760,2021-02-08,2021-09-18,2022-04-11,13483


In [51]:
MarketingCommunication = MarketingCommunication.groupby('CustomerID').agg({
    'Email_Sent': 'count',
    'Email_Opened': 'count',
    'Email_Clicked': 'count'
}).reset_index()
MarketingCommunication.columns = ['CustomerID', 'emails_sent', 'emails_opened', 'emails_clicked']
MarketingCommunication['open_rate'] = MarketingCommunication['emails_opened'] / (MarketingCommunication['emails_sent'] + 1)
MarketingCommunication['click_rate'] = MarketingCommunication['emails_clicked'] / (MarketingCommunication['emails_sent'] + 1)
MarketingCommunication['click_through_rate'] = MarketingCommunication['emails_clicked'] / (MarketingCommunication['emails_opened'] + 1)
MarketingCommunication['marketing_engagement'] = MarketingCommunication['open_rate'] * MarketingCommunication['click_rate']

In [52]:
MarketingCommunication

,CustomerID,emails_sent,emails_opened,emails_clicked,open_rate,click_rate,click_through_rate,marketing_engagement
0,1001,8,8,8,0.888889,0.888889,0.888889,0.790123
1,1002,9,9,9,0.900000,0.900000,0.900000,0.810000
2,1003,8,8,8,0.888889,0.888889,0.888889,0.790123
3,1004,10,10,10,0.909091,0.909091,0.909091,0.826446
4,1005,7,7,7,0.875000,0.875000,0.875000,0.765625
...,...,...,...,...,...,...,...,...
12478,13479,4,4,4,0.800000,0.800000,0.800000,0.640000
12479,13480,7,7,7,0.875000,0.875000,0.875000,0.765625
12480,13481,5,5,5,0.833333,0.833333,0.833333,0.694444
12481,13482,1,1,1,0.500000,0.500000,0.500000,0.250000


### Demographics & Temporal Features

**Idea:** Age, segment, and account age affect churn likelihood.

We create:
- Age groups (Young, Adult, Middle, Senior)
- NPS categories (Detractor, Passive, Promoter)
- Account age in days

In [53]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data['account_age_days'] = (pd.to_datetime('2023-01-01') - data['Timestamp']).dt.days
data['age_group'] = pd.cut(data['Age'], bins=[0, 25, 35, 50, 100], labels=['Young', 'Adult', 'Middle', 'Senior'])
data['nps_category'] = pd.cut(data['NPS'], bins=[-1, 6, 8, 10], labels=['Detractor', 'Passive', 'Promoter'])
demographic_features = data[['CustomerID', 'Age', 'Gender', 'Segment', 'NPS', 'account_age_days', 'age_group', 'nps_category']]

In [54]:
demographic_features

,CustomerID,Age,Gender,Segment,NPS,account_age_days,age_group,nps_category
0,1001,31,Male,Segment B,3,1069,Adult,Detractor
1,1002,66,Female,Segment C,6,1455,Senior,Detractor
2,1003,36,Female,Segment B,3,1341,Middle,Detractor
3,1004,62,Female,Segment C,1,1033,Senior,Detractor
4,1005,68,Female,Segment C,3,1366,Senior,Detractor
...,...,...,...,...,...,...,...,...
12478,13479,55,Female,Segment A,8,338,Senior,Passive
12479,13480,29,Male,Segment A,7,930,Adult,Passive
12480,13481,38,Male,Segment C,1,809,Middle,Detractor
12481,13482,26,Female,Segment A,0,920,Adult,Detractor


### Merge All Features

In [ ]:
data.merge()

In [55]:
main_df = demographic_features.merge(PurchaseHistory, on='CustomerID', how='left') \
    .merge(SubscriptionDetails, on='CustomerID', how='left') \
    .merge(ServiceInteractions, on='CustomerID', how='left') \
    .merge(PaymentHistory, on='CustomerID', how='left') \
    .merge(WebsiteUsage, on='CustomerID', how='left') \
    .merge(ClickstreamData, on='CustomerID', how='left') \
    .merge(EngagementMetrics, on='CustomerID', how='left') \
    .merge(Feedback, on='CustomerID', how='left') \
    .merge(MarketingCommunication, on='CustomerID', how='left')

main_df['ChurnLabel'] = data['ChurnLabel']

In [56]:
main_df

,CustomerID,Age,Gender,Segment,NPS,account_age_days,age_group,nps_category,total_purchase_value,total_frequency,product_diversity,Plan,subscription_duration_days,subscription_age_days,is_active,last_interaction_date,total_interactions,total_late_payments,payment_count,late_payment_rate,payment_risk_score,PageViews,TimeSpent(minutes),engagement_ratio,engagement_intensity,total_actions,add_to_cart_count,search_count,click_count,unique_pages,cart_conversion_rate,search_intensity,page_diversity,Logins,frequency_score,engagement_score,avg_rating,avg_comment_length,sentiment_score,is_negative,is_positive,emails_sent,emails_opened,emails_clicked,open_rate,click_rate,click_through_rate,marketing_engagement,ChurnLabel
0,1001,31,Male,Segment B,3,1069,Adult,Detractor,3994.72,38,7,Express,871,937,0,2021-07-25,4,40,3,10.00,400.00,49,15,0.300000,735,24,8,12,4,13,0.320000,0.480000,0.520000,19,4,76,1.0,96.0,0.2,1,0,8,8,8,0.888889,0.888889,0.888889,0.790123,1
1,1002,66,Female,Segment C,6,1455,Senior,Detractor,2844.35,4,3,Pro,290,529,0,2022-12-13,19,10,3,2.50,25.00,100,9,0.089109,900,24,8,7,9,13,0.320000,0.280000,0.520000,9,4,36,2.0,108.0,0.4,1,0,9,9,9,0.900000,0.900000,0.900000,0.810000,0
2,1003,36,Female,Segment B,3,1341,Middle,Detractor,1866.52,14,3,Essential,319,1184,0,2022-01-04,3,8,3,2.00,16.00,1,97,48.500000,97,12,2,7,3,7,0.153846,0.538462,0.538462,19,1,19,4.0,72.0,0.8,0,1,8,8,8,0.888889,0.888889,0.888889,0.790123,0
3,1004,62,Female,Segment C,1,1033,Senior,Detractor,1378.64,28,5,Smart,803,1083,0,2022-11-10,59,79,3,19.75,1560.25,25,31,1.192308,775,47,15,16,16,14,0.312500,0.333333,0.291667,4,30,120,1.0,78.0,0.2,1,0,10,10,10,0.909091,0.909091,0.909091,0.826446,1
4,1005,68,Female,Segment C,3,1366,Senior,Detractor,2425.05,39,6,Basic,580,633,0,2022-12-19,10,2,3,0.50,1.00,77,51,0.653846,3927,30,17,4,9,12,0.548387,0.129032,0.387097,12,4,48,3.0,99.0,0.6,0,0,7,7,7,0.875000,0.875000,0.875000,0.765625,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12478,13479,55,Female,Segment A,8,338,Senior,Passive,1196.56,14,3,Essential,745,1296,0,2022-10-09,10,3,3,0.75,2.25,70,57,0.802817,3990,6,4,1,1,6,0.571429,0.142857,0.857143,22,30,660,2.0,37.0,0.4,1,0,4,4,4,0.800000,0.800000,0.800000,0.640000,0
12479,13480,29,Male,Segment A,7,930,Adult,Passive,710.57,1,1,Flex,18,22,0,2022-11-05,3,6,3,1.50,9.00,71,66,0.916667,4686,9,3,3,3,8,0.300000,0.300000,0.800000,25,4,100,3.0,102.0,0.6,0,0,7,7,7,0.875000,0.875000,0.875000,0.765625,0
12480,13481,38,Male,Segment C,1,809,Middle,Detractor,5154.42,63,10,Deluxe,20,546,0,2022-12-08,26,83,3,20.75,1722.25,96,1,0.010309,96,26,10,11,5,9,0.370370,0.407407,0.333333,9,1,9,5.0,134.0,1.0,0,1,5,5,5,0.833333,0.833333,0.833333,0.694444,1
12481,13482,26,Female,Segment A,0,920,Adult,Detractor,6055.16,58,9,Gold,484,894,0,2022-11-15,13,67,3,16.75,1122.25,63,2,0.031250,126,38,7,15,16,12,0.179487,0.384615,0.307692,2,1,2,5.0,113.0,1.0,0,1,1,1,1,0.500000,0.500000,0.500000,0.250000,0


#### RFM Analysis

Recency, Frequency and Monetary Analysis = RFM_score

Based on the rfm_score we are segmenting our customers

In [ ]:
3 + 3 + 1 = 7
5 + 1 + 1 = 7
5 + 5 + 3 = 13 = 

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (3446272621.py, line 1)

### Encoding / Feature Encoding

In [31]:
main_df.dtypes

CustomerID                             int64
Age                                    int64
Gender                                object
Segment                               object
NPS                                    int64
account_age_days                       int64
age_group                           category
nps_category                        category
Product                               object
Frequency                              int64
Value                                float64
Plan                                  object
Start_Date                    datetime64[ns]
End_Date                      datetime64[ns]
subscription_duration_days             int64
subscription_age_days                  int64
is_active                              int64
Type                                  object
Date                          datetime64[ns]
total_late_payments                    int64
payment_count                          int64
late_payment_rate                    float64
payment_ri

In [57]:
main_df.select_dtypes(include=['object'])

,Gender,Segment,Plan
0,Male,Segment B,Express
1,Female,Segment C,Pro
2,Female,Segment B,Essential
3,Female,Segment C,Smart
4,Female,Segment C,Basic
...,...,...,...
12478,Female,Segment A,Essential
12479,Male,Segment A,Flex
12480,Male,Segment C,Deluxe
12481,Female,Segment A,Gold


- Label Encoding - 1 2 3 4 5 6 7 8 9 - ki, 
- one hot encoding - 
- target encoding - mean, sum

12483

SEX_BOY SEX_GIRL SEX_LGBTQ  HH NNN
    1        0         0
   0        1         0
   0        1         0
    1        0         0
  1        0         0
  0        1         0
  0        0         1

#### Type of categorical variable

- Nominal CV 
- Ordinal CV